# Inverse methods
For this exercise we will download a dataset of $O_3$ concentration from the TROPOMI instrument on board the Copernicus Sentinel-5p satellite. The dataset can be donloaded from the [Copernicus Dataspace](https://dataspace.copernicus.eu/). In order to download the dataset we have to select an areao of interest, the dates, and the product. From the menu we select: Sentinel-5p -> TROPOMI -> Level-2 O3. The tool can show us the available products. After the download we can use [Panoply](https://www.giss.nasa.gov/tools/panoply/) to have a look at the structure of the dataset.  

In [23]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcol
from matplotlib.colors import ListedColormap
from zipfile import ZipFile
import netCDF4 as nc4

In [6]:
path = 'data/S5P_NRTI_L2_O3_20240818/'
filename = 'S5P_NRTI_L2_O3_20240818.nc'

In [7]:
pwd

'C:\\cygwin64\\home\\Luigi\\eumetsat_training\\cams-act6\\06-ak'

In [14]:
o3_product_ds = xr.open_dataset(path + filename, group='PRODUCT')
o3_product_ds

<xarray.Dataset>
Dimensions:                                (scanline: 357, ground_pixel: 450,
                                            time: 1, corner: 4, layer: 13,
                                            level: 14)
Coordinates:
  * scanline                               (scanline) float64 0.0 1.0 ... 356.0
  * ground_pixel                           (ground_pixel) float64 0.0 ... 449.0
  * time                                   (time) datetime64[ns] 2024-08-18
  * corner                                 (corner) float64 0.0 1.0 2.0 3.0
  * layer                                  (layer) int32 0 1 2 3 ... 9 10 11 12
  * level                                  (level) int32 0 1 2 3 ... 10 11 12 13
Data variables:
    latitude                               (time, scanline, ground_pixel) float32 ...
    longitude                              (time, scanline, ground_pixel) float32 ...
    delta_time                             (time, scanline, ground_pixel) datetime64[ns] ...
    time_utc                               (time, scanline) object ...
    qa_value                               (time, scanline, ground_pixel) float32 ...
    ozone_total_vertical_column            (time, scanline, ground_pixel) float32 ...
    ozone_total_vertical_column_precision  (time, scanline, ground_pixel) float32 ...

In [25]:
from netCDF4 import Dataset
fh = Dataset(path + filename, mode='r')
fh

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: DLR-IMF
    source: Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
    history: 2024-08-18 12:49:05.739787 f_s5pops upas-l2 JobOrder.674317873.xml 
    summary: TROPOMI/S5P L2 data Swath 5.5x3.5km2 processed in NRTI mode
    tracking_id: debfd6e9-b359-4f36-9f33-643cdb9e5b0b
    id: S5P_NRTI_L2__O3_____20240818T120300_20240818T120800_35484_03_020601_20240818T124850
    time_reference: 2024-08-18T00:00:00Z
    time_reference_days_since_1950: 27258
    time_reference_julian_day: 2460540.5
    time_reference_seconds_since_1970: 1723939200
    time_coverage_start: 2024-08-18T12:02:54Z
    time_coverage_end: 2024-08-18T12:08:07Z
    time_coverage_duration: PT313.000S
    time_coverage_resolution: PT0.840S
    orbit: 35484
    references: TBD
    processor_version: 02.06.01
    keywords_vocabulary: AGU index terms, http://publications.agu.org/author-res

In [28]:
o3_support_ds = xr.open_dataset(path + filename, group='PRODUCT/SUPPORT_DATA/DETAILED_RESULTS')
o3_support_ds.keys()

KeysView(<xarray.Dataset>
Dimensions:                                 (time: 1, scanline: 357,
                                             ground_pixel: 450, layer: 13,
                                             level: 14,
                                             number_of_slant_columns: 4,
                                             number_of_doas_polynomial_coefficients: 4)
Coordinates:
  * number_of_slant_columns                 (number_of_slant_columns) int32 0...
  * number_of_doas_polynomial_coefficients  (number_of_doas_polynomial_coefficients) int32 ...
Dimensions without coordinates: time, scanline, ground_pixel, layer, level
Data variables: (12/25)
    ozone_profile_apriori                   (time, scanline, ground_pixel, layer) float32 ...
    averaging_kernel                        (time, scanline, ground_pixel, layer) float32 ...
    pressure_grid                           (time, scanline, ground_pixel, level) float32 ...
    fitted_slant_columns                   